# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import decimal
from IPython.display import HTML

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
#print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(*file_path_list, sep="\n")

/workspace/home/event_data/2018-11-23-events.csv
/workspace/home/event_data/2018-11-17-events.csv
/workspace/home/event_data/2018-11-04-events.csv
/workspace/home/event_data/2018-11-11-events.csv
/workspace/home/event_data/2018-11-18-events.csv
/workspace/home/event_data/2018-11-20-events.csv
/workspace/home/event_data/2018-11-10-events.csv
/workspace/home/event_data/2018-11-16-events.csv
/workspace/home/event_data/2018-11-27-events.csv
/workspace/home/event_data/2018-11-14-events.csv
/workspace/home/event_data/2018-11-25-events.csv
/workspace/home/event_data/2018-11-29-events.csv
/workspace/home/event_data/2018-11-30-events.csv
/workspace/home/event_data/2018-11-06-events.csv
/workspace/home/event_data/2018-11-13-events.csv
/workspace/home/event_data/2018-11-15-events.csv
/workspace/home/event_data/2018-11-03-events.csv
/workspace/home/event_data/2018-11-02-events.csv
/workspace/home/event_data/2018-11-22-events.csv
/workspace/home/event_data/2018-11-21-events.csv
/workspace/home/even

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace T
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS t 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('t')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [8]:
## Create queries to ask the following three questions of the data
### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [9]:
## Query 1: Give me the artist, song title and song's length in the music app history that was heard during \
##          sessionId = 338, and itemInSession = 4
## 'SELECT artist, song, length FROM music_app_history WHERE itemInSession=4 and sessionid=338'
## Using column "iteminsession" as the partition key will distribute data items equaly in nodes for small amount of session items
## Using clustered column "sessionid" will allow us to use it in the WHERE clause and insure the primary key is unique

query1 = "CREATE TABLE IF NOT EXISTS music_app_artist " 
query1 = query1 + "(iteminsession int, sessionid int, artist text, song text, length decimal, PRIMARY KEY(iteminsession, sessionid))"
try:
    session.execute(query1)
except Exception as e:
    print(e)       

In [10]:
file = 'event_datafile_new.csv'
##INSERT statements
query = "INSERT INTO music_app_artist (iteminsession, sessionid, artist, song, length)"  
query = query + " VALUES (%s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Column element assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[3]), int(line[8]), line[0], line[9], decimal.Decimal(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
# SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, length FROM music_app_artist WHERE iteminsession = 4 and sessionid=338"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df = pd.DataFrame(list(rows))
HTML(df.to_html(index=False))

artist,song,length
Faithless,Music Matters (Mark Knight Dub),495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [14]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
## 'SELECT artist, song, user FROM music_app_user WHERE userid=10 and sessionid=182'
## Partition is by "user" on nodes for our small amount of data and 
## clustered columns "sessionid" and "iteminsession" insure a unique primary key and can be used in the WHERE clause
## Clustered columns "sessionid" and "iteminsession" are ordered ascending by declaration order, so we are satisfying
## the "iteminsession" ordering request.

query2 = "CREATE TABLE IF NOT EXISTS music_app_user " 
query2 = query2 + "(userid int, sessionid int, iteminsession int, artist text, song text, user text, PRIMARY KEY((userid),sessionid, iteminsession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)       

                    

In [15]:
file = 'event_datafile_new.csv'
##INSERT statements
query = "INSERT INTO music_app_user (userid, sessionid, iteminsession, artist, song, user)"  
query = query + " VALUES (%s, %s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Column element assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], (line[1]+' '+ line[4])))

In [17]:
# SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, user FROM music_app_user WHERE userid=10 and sessionid=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame(list(rows))
HTML(df.to_html(index=False))

artist,song,user
Down To The Bone,Keep On Keepin' On,Sylvie Cruz
Three Drives,Greece 2000,Sylvie Cruz
Sebastien Tellier,Kilometer,Sylvie Cruz
Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


In [20]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## SELECT user WHERE song = 'All Hands Against His Own'
## Partition is by "song" on nodes for our small amount of data and 
## clustered columns "userid" and "sessionid" insure a unique primary key for the specific query 3 
## and can be used in the WHERE clause

query3 = "CREATE TABLE IF NOT EXISTS music_app_song " 
query3 = query3 + "(song text, userid int, sessionid int, user text, PRIMARY KEY((song), userid, sessionid))"
try:
    session.execute(query3)
except Exception as e:
    print(e)       

                    

In [21]:
file = 'event_datafile_new.csv'
##INSERT statements
query = "INSERT INTO music_app_song (song, userid, sessionid, user)"  
query = query + " VALUES (%s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Column element assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), int(line[8]), (line[1]+' '+ line[4])))

In [22]:
# SELECT statement to verify the data was entered into the table
query = "SELECT user FROM music_app_song WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame(list(rows))
HTML(df.to_html(index=False))

user
Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [23]:
## Drop the table before closing out the sessions
try:
   session.execute('DROP TABLE music_app_artist')
   session.execute('DROP TABLE music_app_user')
   session.execute('DROP TABLE music_app_song')
except Exception as e:
    print(e)       

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()